In [49]:
def tprint(n,i,s=""):
    print("\n=== Test",i,"===")
    if s!="": print(s)
    print(n,"\nsize",n.getSize())

net = NetworkWithUndo(9)
tprint(net,0)
net.merge(4,0); net.merge(0,3); net.merge(2,1); net.merge(6,8); net.merge(8,7)
tprint(net,1)
net.merge(0,8)
tprint(net,2)
x = net.root(4)
tprint(net,3,"root of 4: "+str(x))
net.undo()
tprint(net,4)
net.undo()
tprint(net,5)
net.merge(0,1)
tprint(net,6)
for i in range(8): 
    net.undo(); tprint(net,i+6)


=== Test 0 ===
[-1, -1, -1, -1, -1, -1, -1, -1, -1]
-> [9] 
size 9

=== Test 1 ===
[4, 2, -1, 4, -1, -1, -1, 6, 6]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 2 ===
[4, 2, -1, 4, -1, -1, 4, 6, 6]
-> [2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 3 ===
root of 4: 4
[4, 2, -1, 4, -1, -1, 4, 6, 6]
-> [2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 4 ===
[4, 2, -1, 4, -1, -1, -1, 6, 6]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 5 ===
[4, 2, -1, 4, -1, -1, -1, 6, 6]
-> [2, 2, 2, 2, 9] 
size 9

=== Test 6 ===
[4, 2, 4, 4, -1, -1, -1, 6, 6]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 6 ===
[4, 2, -1, 4, -1, -1, -1, 6, 6]
-> [2, 2, 2, 2, 9] 
size 9

=== Test 7 ===
[4, 2, -1, 4, -1, -1, -1, 6, 6]
-> [2, 2, 2, 9] 
size 9

=== Test 8 ===
[4, 2, -1, 4, -1, -1, -1, -1, 6]
-> [2, 2, 9] 
size 9

=== Test 9 ===
[4, 2, -1, 4, -1, -1, -1, -1, 6]
-> [2, 9] 
size 9

=== Test 10 ===
[4, 2, -1, 4, -1, -1, -1, -1, -1]
-> [9] 
size 9

=== Test 11 ===
[-1, -1, -1, -1, -1, -1]
-> [] 
size 6

=== Test 12 ===
[-1, -1, -1, -1, -1, -1]
-> 

In [48]:
class LinkedList:
    def __init__(self):
        self.head = None
        self.length = 0

    def search(self, d):
        i = 0
        ptr = self.head
        while ptr is not None:
            if ptr.data == d:
                return i
            ptr = ptr.next
            i += 1
        return -1

    def append(self, d):
        if self.head is None:
            self.head = Node(d, None)
        else:
            ptr = self.head
            while ptr.next is not None:
                ptr = ptr.next
            ptr.next = Node(d, None)
        self.length += 1

    def insert(self, i, d):
        if self.head is None or i == 0:
            self.head = Node(d, self.head)
        else:
            ptr = self.head
            while i > 1 and ptr.next is not None:
                ptr = ptr.next
                i -= 1
            ptr.next = Node(d, ptr.next)
        self.length += 1

    def remove(self, i):  # removes/returns i-th element
        if self.head is None:
            return None
        if i == 0:
            val = self.head.data
            self.head = self.head.next
            self.length -= 1
            return val
        else:
            ptr = self.head
            while i > 1 and ptr.next is not None:
                ptr = ptr.next
                i -= 1
            if i == 1 and ptr.next is not None:
                val = ptr.next.data
                ptr.next = ptr.next.next
                self.length -= 1
                return val
            return None

In [47]:
class NetworkWithUndo:
    def __init__(self, N):
        self.inArray = ArrayListWithUndo()
        for _ in range(N):
            self.inArray.append(-1)  # Each node starts as its own cluster (size -1)
        self.undos = Stack()
        self.undos.push(N)  # Initial state

    def getSize(self):
        return self.inArray.length()

    def add(self):
        n_operations = 0
        new_node = self.getSize()
        self.inArray.append(-1)  # New node as its own cluster
        n_operations += 1
        self.undos.push(n_operations)

    def root(self, i):
        visited_nodes = []  # To store the path for path flattening
        while self.inArray.get(i) >= 0:  # While i is not a root
            visited_nodes.append(i)
            i = self.inArray.get(i)
    
        # Path flattening: make all visited nodes point directly to the root
        for node in visited_nodes:
            self.inArray.set(node, i)
    
        return i

    def merge(self, i, j):
        # Find the roots of i and j
        root_i = self.root(i)  # Apply path flattening
        root_j = self.root(j)  # Apply path flattening
    
        # Assert both i and j are root nodes (they should be the roots of their respective clusters)
        assert self.inArray.get(root_i) < 0, "i is not a root"
        assert self.inArray.get(root_j) < 0, "j is not a root"
    
        # If they are already in the same cluster, do nothing
        if root_i == root_j:
            return
    
        # Get the sizes of the clusters rooted at i and j
        size_i = -self.inArray.get(root_i)  # Size of the cluster rooted at i
        size_j = -self.inArray.get(root_j)  # Size of the cluster rooted at j
        n_operations = 0
    
        # Merge smaller cluster into the larger one
        if size_i < size_j:
            self.inArray.set(root_i, root_j)  # root_i points to root_j
            self.inArray.set(root_j, -size_i)  # Update size of cluster rooted at root_j
            n_operations += 2
        else:
            self.inArray.set(root_j, root_i)  # root_j points to root_i
            self.inArray.set(root_i, -size_j)  # Update size of cluster rooted at root_i
            n_operations += 2
    
        self.undos.push(n_operations)  # Push the number of operations to undo


    def undo(self):
        # If no operations to undo, return
        if self.undos.isEmpty():
            return

        # Get the number of operations to undo
        n_operations = self.undos.pop()

        # Perform n undo operations on the underlying ArrayListWithUndo
        for _ in range(n_operations):
            self.inArray.undo()

    def toArray(self):
        return self.inArray.toArray()

    def __str__(self):
        return str(self.toArray()) + "\n-> " + str(self.undos)


In [46]:
# Implementation of a simple Stack and an ArrayList with undo functionality.

class Stack:
    """A simple stack implementation using a linked list representation."""
    
    def __init__(self):
        # Initialize an empty stack with size 0 and no elements.
        self.inList = None
        self.size = 0
        
    def push(self, v):
        """Push a value onto the stack."""
        self.size += 1
        # Store the new value and link it to the current stack.
        self.inList = (v, self.inList)
        
    def pop(self):
        """Pop the top value from the stack."""
        # Assert ensures the stack is not empty before popping.
        if self.size == 0: assert(0)
        self.size -= 1
        # Retrieve the top value and update the stack pointer.
        (v, ls) = self.inList
        self.inList = ls
        return v
    
    def isEmpty(self):
        """Check if the stack is empty."""
        return self.size == 0
    
    def size(self):
        return len(self.stack)
    
    def __str__(self):
        """Return a string representation of the stack."""
        s = "["
        ls = self.inList
        for _ in range(self.size):
            (v, ls) = ls
            s += str(v)
            if ls != None: s += ", "
        return s + "]"
        
class ArrayList:
    """A dynamic array-like data structure."""
    
    def __init__(self):
        # Initialize with a fixed capacity array and count of elements.
        self.inArray = [0 for _ in range(10)]
        self.count = 0
        
    def get(self, i):
        """Get the element at index i."""
        return self.inArray[i]

    def set(self, i, e):
        """Set the element at index i to e."""
        self.inArray[i] = e

    def length(self):
        """Return the current number of elements."""
        return self.count

    def append(self, e):
        """Add an element to the end of the array."""
        self.inArray[self.count] = e
        self.count += 1
        # Resize the array if capacity is reached.
        if len(self.inArray) == self.count:
            self._resizeUp()

    def insert(self, i, e):
        """Insert an element at index i, shifting subsequent elements."""
        for j in range(self.count, i, -1):
            self.inArray[j] = self.inArray[j-1]
        self.inArray[i] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()
    
    def remove(self, i):
        """Remove the element at index i and shift subsequent elements."""
        self.count -= 1
        val = self.inArray[i]
        for j in range(i, self.count):
            self.inArray[j] = self.inArray[j+1]
        return val

    def _resizeUp(self):
        """Double the array size when capacity is exceeded."""
        newArray = [0 for _ in range(2 * len(self.inArray))]
        for j in range(len(self.inArray)):
            newArray[j] = self.inArray[j]
        self.inArray = newArray
        
    def toArray(self):
        """Return a list representation of the current elements."""
        return self.inArray[:self.count]

    def __str__(self):
        """Return a string representation of the array."""
        if self.count == 0: return "[]"
        s = "["
        for i in range(self.count-1): s += str(self.inArray[i]) + ", "
        return s + str(self.inArray[self.count-1]) + "]"

class ArrayListWithUndo(ArrayList):
    """Extension of ArrayList to support undo operations."""
    
    def __init__(self):
        # Initialize ArrayList and a stack for undo operations.
        super().__init__()
        self.undos = Stack()

    def set(self, i, v):
        """Set element at index i and store undo information."""
        self.undos.push(("set", i, self.inArray[i]))
        self.inArray[i] = v

    def append(self, v):
        """Append an element and store undo information."""
        self.undos.push(("rem", self.count, None))
        super().append(v)

    def remove(self, i):
        """Remove element at index i and store undo information."""
        if i < self.count:  # Check if i is within bounds
            self.undos.push(("ins", i, self.inArray[i]))
            super().remove(i)

    def insert(self, i, v):
        """Insert element at index i and store undo information."""
        if i <= self.count:  # Ensure valid index
            self.undos.push(("rem", i, None))
            super().insert(i, v)

    def undo(self):
        """Undo the last operation."""
        if self.undos.size == 0:
            return

        op, i, v = self.undos.pop()

        if op == "set":
            self.inArray[i] = v  # Restore previous value
        elif op == "rem":
            if self.count == 0:
                self.count = 0
            else:
                self.count -= 1  # Undo an append operation
        elif op == "ins":
            self.insert(i, v)  # Reinsert a removed element

    def __str__(self):
        """Return string representation of the list and undo stack."""
        return str(self.toArray()) + "\n-> " + str(self.undos)

# Minimal tests for ArrayListWithUndo

def tprint(ls, i):
    """Print test case results."""
    print("\n=== Test", i, "===\n", ls, "\nsize", ls.length())

ls = ArrayListWithUndo()
A = [2, 3, 4, 5, 5, 1, 4]
for x in A: 
    ls.append(x)
ls.set(4, 2)
ls.insert(3, 10)
ls.remove(0)
tprint(ls, 0)
for i in range(len(A) + 4):
        ls.undo()
        tprint(ls, i + 1)



=== Test 0 ===
 [3, 4, 10, 5, 2, 1, 4]
-> [('ins', 0, 2), ('rem', 3, None), ('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 7

=== Test 1 ===
 [2, 3, 4, 10, 5, 2, 1, 4]
-> [('rem', 0, None), ('rem', 3, None), ('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 8

=== Test 2 ===
 [2, 3, 4, 10, 5, 2, 1]
-> [('rem', 3, None), ('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 7

=== Test 3 ===
 [2, 3, 4, 10, 5, 2]
-> [('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 6

=== Test 4 ===
 [2, 3, 4, 10, 5, 2]
-> [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1,